In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/predict-test-scores-of-students/test_scores.csv")
df.head()

In [ ]:
df.drop(["student_id","classroom"], axis= 1, inplace= True)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
for d in df.columns:
    print("{0} : {1}".format(d, df[d].nunique()))

In [ ]:
categories = ['school','school_setting', 'school_type', 'teaching_method', 'gender', 'lunch']
df_origin = pd.get_dummies(df, columns=categories, drop_first=True)
df_origin.head()

In [ ]:
from sklearn.model_selection import train_test_split

X = df_origin.drop('posttest', axis=1).values
y = df_origin['posttest'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluation(y_true, y_pred):
    print("R2 score: ", r2_score(y_true, y_pred))
    print("Mean Absolute Error: ", mean_absolute_error(y_true, y_pred))
    print("Mean Squared Error: ", mean_squared_error(y_true, y_pred))

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
evaluation(y_true=y_test, y_pred = lr.predict(X_test))

In [ ]:
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.metrics import mean_squared_error

ridge_cv = RidgeCV(alphas=np.logspace(-10, 6, 20))
ridge_cv.fit(X_train, y_train)
ridge = Ridge(alpha = ridge_cv.alpha_)
ridge.fit(X_train, y_train)
evaluation(y_true=y_test, y_pred = ridge.predict(X_test))

In [ ]:
from sklearn.svm import SVR

svr_reg = SVR(C=2.5, epsilon=0.3, kernel='rbf')
svr_reg.fit(X_train, y_train)
evaluation(y_true=y_test, y_pred = svr_reg.predict(X_test))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=1000, min_samples_leaf=2, min_samples_split=6, max_features='sqrt', max_depth=30)
rf.fit(X_train, y_train)
evaluation(y_true=y_test, y_pred = rf.predict(X_test))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor( n_estimators=1000, learning_rate=0.1, max_depth=30,min_samples_split=6, min_samples_leaf=2, max_features='sqrt')
gbr.fit(X_train, y_train)
evaluation(y_true=y_test, y_pred = gbr.predict(X_test))